# INFO 323: Cloud Computing and Big Data
## CCI at Drexel University
### Yuan An

## Check the Apache Spark Availability and Version

In [2]:
sc.version

'2.4.8'

In [1]:
spark.version

'2.4.8'

### DataFrame and RDD

In [13]:
spark.range(10).toDF("id").rdd.map(lambda row: row[0]).collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

## Spark Example: Word Count: Access Data from Google File System
### 1. We use sparkContext object to read a text file as records of Resilient Distributed Dataset (RDD). Each line is a record. RDD is the primitive data structure used by Spark. 

In [18]:
file = spark.sparkContext.textFile("gs://info323-ya45-bucket/notebooks/jupyter/words.txt")

In [19]:
type(file)

pyspark.rdd.RDD

In [22]:
file.take(5)

['The Project Gutenberg EBook of A Tale of Two Cities, by Charles Dickens',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included']

### 2. Transform the RDD into records of individual words
We will use the flatMap() operation to convert the records of lines to records of individual words. 

Mapping is a basic operation for transforming RDD records. We specify a function
that returns the value that we want, given the correct input. We then apply that, record by record.

Because we want the words from all lines reside in the same list, not in nested lists, we apply flatMap() which flatten nested lists.

Compare the results of map() and flatMap()

In [25]:
words = file.map(lambda line: line.split())

In [26]:
words.take(3)

[['The',
  'Project',
  'Gutenberg',
  'EBook',
  'of',
  'A',
  'Tale',
  'of',
  'Two',
  'Cities,',
  'by',
  'Charles',
  'Dickens'],
 [],
 ['This',
  'eBook',
  'is',
  'for',
  'the',
  'use',
  'of',
  'anyone',
  'anywhere',
  'at',
  'no',
  'cost',
  'and',
  'with']]

In [27]:
words = file.flatMap(lambda line: line.split())

In [28]:
words.take(3)

['The', 'Project', 'Gutenberg']

### 3. Transform the records of individual words to records of word-count_1 pairs
We apply the map() operation to the records of words and output word-1 pairs

In [29]:
word_1 = words.map(lambda word: (word, 1))

In [30]:
word_1.take(5)

[('The', 1), ('Project', 1), ('Gutenberg', 1), ('EBook', 1), ('of', 1)]

### 4. Reduce the records of word_1 pairs to records of word_count pairs

We can use the reduce() method to specify a function to “reduce” an RDD of any kind of value to one
value. For instance, given a set of numbers, we can reduce this to its sum by specifying a function that
takes as input two values and reduces them into one. 

For the word count application, we use reduceByKey() to count different words.

The reduce() method is an action which kicks off the specified transformations. It returns the final result.

However, reduceByKey() is a transformation which transforms one RDD to another RDD. To kick off the transformations, an action is required such as collect() or take().

Compare the results of reduce() and reduceByKey().

In [35]:
counts_total = word_1.reduce(lambda a, b: a+b)

In [36]:
counts_total[:5]

('The', 1, 'Project', 1, 'Gutenberg')

In [37]:
counts = word_1.reduceByKey(lambda a, b: a+b)

In [39]:
counts.take(5)

[('The', 587), ('Project', 78), ('EBook', 2), ('of', 4065), ('Tale', 3)]

### 5. Save the results which may be in multiple partitions
Specify a new folder to save the results. If the folder name exists, error messages may appear.

In [40]:
counts.saveAsTextFile("gs://info323-ya45-bucket/notebooks/jupyter/counts-spark")

### 6. Go back to the storage to download the partitions and assemble the results
You may need to download the partitions to a local system and combine the partitions to a single file.

## Spark Example: Word Count: Access Data from HDFS
### 1. We use sparkContext object to read a text file as records of Resilient Distributed Dataset (RDD). Each line is a record. RDD is the primitive data structure used by Spark. 

In [2]:
file = spark.sparkContext.textFile("hdfs:/user/words/words.txt")

In [3]:
type(file)

pyspark.rdd.RDD

In [4]:
file.take(5)

['The Project Gutenberg EBook of A Tale of Two Cities, by Charles Dickens',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included']

### 2. Transform the RDD into records of individual words
We will use the flatMap() operation to convert the records of lines to records of individual words. 

Mapping is a basic operation for transforming RDD records. We specify a function
that returns the value that we want, given the correct input. We then apply that, record by record.

Because we want the words from all lines reside in the same list, not in nested lists, we apply flatMap() which flatten nested lists.

Compare the results of map() and flatMap()

In [5]:
words = file.map(lambda line: line.split())

In [6]:
words.take(3)

[['The',
  'Project',
  'Gutenberg',
  'EBook',
  'of',
  'A',
  'Tale',
  'of',
  'Two',
  'Cities,',
  'by',
  'Charles',
  'Dickens'],
 [],
 ['This',
  'eBook',
  'is',
  'for',
  'the',
  'use',
  'of',
  'anyone',
  'anywhere',
  'at',
  'no',
  'cost',
  'and',
  'with']]

In [7]:
words = file.flatMap(lambda line: line.split())

In [8]:
words.take(3)

['The', 'Project', 'Gutenberg']

### 3. Transform the records of individual words to records of word-count_1 pairs
We apply the map() operation to the records of words and output word-1 pairs

In [9]:
word_1 = words.map(lambda word: (word, 1))

In [10]:
word_1.take(5)

[('The', 1), ('Project', 1), ('Gutenberg', 1), ('EBook', 1), ('of', 1)]

### 4. Reduce the records of word_1 pairs to records of word_count pairs

We can use the reduce() method to specify a function to “reduce” an RDD of any kind of value to one
value. For instance, given a set of numbers, we can reduce this to its sum by specifying a function that
takes as input two values and reduces them into one. 

For the word count application, we use reduceByKey() to count different words.

The reduce() method is an action which kicks off the specified transformations. It returns the final result.

However, reduceByKey() is a transformation which transforms one RDD to another RDD. To kick off the transformations, an action is required such as collect() or take().

Compare the results of reduce() and reduceByKey().

In [11]:
counts_total = word_1.reduce(lambda a, b: a+b)

In [12]:
counts_total[:5]

('The', 1, 'Project', 1, 'Gutenberg')

In [13]:
counts = word_1.reduceByKey(lambda a, b: a+b)

In [14]:
counts.take(5)

[('The', 587), ('Project', 78), ('EBook', 2), ('of', 4065), ('Tale', 3)]

### 5. Save the results which may be in multiple partitions
Specify a new folder to save the results. If the folder name exists, error messages may appear.

In [15]:
counts.saveAsTextFile("hdfs:/user/word-counts")

### 6. Go back to the storage to download the partitions and assemble the results
You may need to download the partitions to a local system and combine the partitions to a single file.

# Pi: Compute an approximate value of $\pi$
The area of square of side 2 is 4. The area of the circle inside the square is $\pi \times 1^2 = \pi$. So $\frac{\pi}{4}\approx \frac{\textrm{no. of points generated inside the circle}}{\textrm{no. of points generated inside the square}} $.

$\pi \approx 4 \times \frac{\textrm{no. of points generated inside the circle}}{\textrm{no. of points generated inside the square}}$.

### 1. Define a utility function

In [11]:
def inside(p):
    x, y = random.random(), random.random()
    return x**2 + y**2 < 1

### 2. Compute the number of points in the circle

In [12]:
import random
NUM_SAMPLES = 1000000
count = sc.parallelize(range(0, NUM_SAMPLES)).filter(inside).count()
print(count)

785211


### 3. Approximate $\pi$

In [13]:
print("Pi is roughly %f" % (4.0 * count / NUM_SAMPLES))

Pi is roughly 3.140844
